In [1]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import json 

In [2]:
# add labels 
# integerization of compound id 
# integerization and mapping of the assay id 
# T2, T0 creation 
# --- 

In [3]:
t1 = pd.read_csv('./files/T1_image_pseudolabel_aux_nolabels.csv')

In [4]:
t1

,standard_value,input_compound_id,standard_qualifier,input_assay_id
0,0,2255056,=,8
1,0,874219,=,8
2,0,1013304,=,8
3,0,764449,=,8
4,0,783209,=,8
...,...,...,...,...
457468,1,2275364,=,280
457469,1,743980,=,280
457470,1,770512,=,280
457471,1,2275365,=,280


In [5]:
## T0 creation

In [6]:
path = '../../ci8b00670_si_002/Label_Matrix/T_files/all/T0.csv'
melloddy_t0 = pd.read_csv(path)

In [7]:
T10c_cont_hofmarcher = pd.read_csv('../../data/Mellody_tuner/images/output_images/pseudolabels/results/T10c_cont.csv')
T10c_cont_hofmarcher = T10c_cont_hofmarcher[['input_assay_id', 'cont_classification_task_id', 'threshold' ]].drop_duplicates()


In [9]:
T10c_cont_baseline = pd.read_csv('../../data/Mellody_tuner/all/output_all/pseudolabels_w_melloddy/results/T10c_cont.csv')
T10c_cont_baseline = T10c_cont_baseline[T10c_cont_baseline.input_assay_id.isin(T10c_cont_hofmarcher.input_assay_id.unique())]
T10c_cont_baseline = T10c_cont_baseline[['input_assay_id', 'cont_classification_task_id', 'threshold' ]].drop_duplicates()



In [12]:
T10c_cont_hofmarcher[['input_assay_id', 'cont_classification_task_id', 'threshold' ]]


,input_assay_id,cont_classification_task_id,threshold
0,600886,0,0.0
131,688111,1,4.7
304,688113,2,3.9
396,688113,3,4.4
488,688125,4,4.2
...,...,...,...
183622,4000027,280,0.0
184398,4000029,281,0.0
184557,4000030,282,0.0
184645,4000031,283,0.0


In [13]:
T10c_cont_baseline[['input_assay_id', 'cont_classification_task_id', 'threshold']]

,input_assay_id,cont_classification_task_id,threshold
43365,600886,448,0.0
55473,688111,533,4.7
58616,688111,534,5.2
61759,688113,535,4.2
63424,688113,536,4.7
...,...,...,...
3680175,4000027,2945,0.0
3680951,4000029,2946,0.0
3681110,4000030,2947,0.0
3681198,4000031,2948,0.0


In [15]:
mapping_df = pd.merge(T10c_cont_baseline,T10c_cont_hofmarcher, on=['input_assay_id', 'threshold'])

In [18]:
mapping_df

,input_assay_id,cont_classification_task_id_x,threshold,cont_classification_task_id_y
0,600886,448,0.0,0
1,688111,533,4.7,1
2,688138,546,4.4,10
3,688141,548,4.2,11
4,688141,549,4.7,12
...,...,...,...,...
220,4000027,2945,0.0,280
221,4000029,2946,0.0,281
222,4000030,2947,0.0,282
223,4000031,2948,0.0,283


In [16]:
assays_to_keep = mapping_df[mapping_df.cont_classification_task_id_y.isin(t1.input_assay_id.unique())]
image_iai_cont_map_2 = assays_to_keep[['cont_classification_task_id_y', 'cont_classification_task_id_x']].set_index('cont_classification_task_id_y').to_dict()['cont_classification_task_id_x']


In [17]:
image_iai_cont_map_2

{11: 548,
 12: 549,
 14: 558,
 16: 561,
 17: 562,
 21: 569,
 23: 571,
 24: 572,
 30: 609,
 31: 610,
 40: 631,
 41: 639,
 53: 680,
 56: 683,
 59: 686,
 65: 698,
 66: 705,
 70: 728,
 71: 729,
 72: 730,
 86: 770,
 87: 771,
 88: 772,
 101: 802,
 112: 822,
 113: 823,
 118: 832,
 119: 833,
 138: 869,
 144: 888,
 146: 890,
 149: 900,
 150: 901,
 151: 902,
 157: 915,
 158: 916,
 159: 917,
 162: 927,
 164: 934,
 174: 947,
 176: 949,
 177: 950,
 190: 1011,
 193: 1070,
 195: 1106,
 200: 1165,
 203: 1168,
 206: 1172,
 218: 1212,
 220: 1214,
 224: 1228,
 230: 1256,
 244: 1701,
 248: 1708,
 250: 1710,
 263: 1898,
 276: 2941,
 277: 2942,
 278: 2943,
 279: 2944,
 280: 2945}

In [237]:
with open('./files/image_iai_cont_map_2.json', 'w') as fp:
    json.dump(image_iai_cont_map_2, fp)

In [9]:
# change from AUX_IMG to AUX_HTS 
# change the 1.0 label to 6.0 (to accommodate the rscore 3 threshold and force positive labels)
# remove the 0.5 threshold from T0 


In [22]:
n_image_id_start = 10000000 #(high number)
image_iai_cont_map = {e:i+n_image_id_start for i,e in enumerate(image_iai_cont_map_2.keys())}

In [23]:
image_iai_cont_map

{11: 10000000,
 12: 10000001,
 14: 10000002,
 16: 10000003,
 17: 10000004,
 21: 10000005,
 23: 10000006,
 24: 10000007,
 30: 10000008,
 31: 10000009,
 40: 10000010,
 41: 10000011,
 53: 10000012,
 56: 10000013,
 59: 10000014,
 65: 10000015,
 66: 10000016,
 70: 10000017,
 71: 10000018,
 72: 10000019,
 86: 10000020,
 87: 10000021,
 88: 10000022,
 101: 10000023,
 112: 10000024,
 113: 10000025,
 118: 10000026,
 119: 10000027,
 138: 10000028,
 144: 10000029,
 146: 10000030,
 149: 10000031,
 150: 10000032,
 151: 10000033,
 157: 10000034,
 158: 10000035,
 159: 10000036,
 162: 10000037,
 164: 10000038,
 174: 10000039,
 176: 10000040,
 177: 10000041,
 190: 10000042,
 193: 10000043,
 195: 10000044,
 200: 10000045,
 203: 10000046,
 206: 10000047,
 218: 10000048,
 220: 10000049,
 224: 10000050,
 230: 10000051,
 244: 10000052,
 248: 10000053,
 250: 10000054,
 263: 10000055,
 276: 10000056,
 277: 10000057,
 278: 10000058,
 279: 10000059,
 280: 10000060}

In [213]:
t0_image = pd.DataFrame.from_dict(image_iai_cont_map,orient='index').reset_index()
t0_image = t0_image.rename(columns={'index':'melloddy_col_idx',0:'input_assay_id'})

In [214]:
t0_image['source'] = 'image-based pseudolabels'
t0_image['assay_type'] =  'AUX_HTS'
t0_image['assay_type_reason'] = 'image-based pseudolabels'
t0_image['model_description'] = 'img_' + t0_image['melloddy_col_idx'].astype(str)
t0_image['assay_description'] = 'img_' + t0_image['melloddy_col_idx'].astype(str)


In [215]:
t0_image

,melloddy_col_idx,input_assay_id,source,assay_type,assay_type_reason,model_description,assay_description
0,11,10000000,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_11,img_11
1,12,10000001,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_12,img_12
2,14,10000002,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_14,img_14
3,16,10000003,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_16,img_16
4,17,10000004,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_17,img_17
...,...,...,...,...,...,...,...
56,276,10000056,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_276,img_276
57,277,10000057,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_277,img_277
58,278,10000058,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_278,img_278
59,279,10000059,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_279,img_279


In [216]:
t0_image['melloddy_col_idx'] = t0_image['melloddy_col_idx'].map(image_iai_cont_map_2)

In [217]:
t0_image

,melloddy_col_idx,input_assay_id,source,assay_type,assay_type_reason,model_description,assay_description
0,548,10000000,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_11,img_11
1,549,10000001,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_12,img_12
2,558,10000002,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_14,img_14
3,561,10000003,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_16,img_16
4,562,10000004,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_17,img_17
...,...,...,...,...,...,...,...
56,2941,10000056,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_276,img_276
57,2942,10000057,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_277,img_277
58,2943,10000058,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_278,img_278
59,2944,10000059,image-based pseudolabels,AUX_HTS,image-based pseudolabels,img_279,img_279


In [218]:
path = './files/T0_image_pseudolabels_full.csv'
t0_image.to_csv(path,index=False)

In [78]:
# T1 export 

In [228]:
t1['standard_value'] = np.where(t1['standard_value']==1,6,0)

In [229]:
t1['input_assay_id_melloddy'] = t1['input_assay_id'].map(image_iai_cont_map)

In [230]:
#t1['input_compound_id_melloddy'] = t1['input_compound_id'].map(compound_integerization_map)
t1=t1.dropna()


In [231]:
t1

,standard_value,input_compound_id,standard_qualifier,input_assay_id,input_assay_id_melloddy
4484,6,970891,=,11,10000000.0
4485,6,2255064,=,11,10000000.0
4486,0,907121,=,11,10000000.0
4487,6,2255077,=,11,10000000.0
4488,0,34005,=,11,10000000.0
...,...,...,...,...,...
457468,6,2275364,=,280,10000060.0
457469,6,743980,=,280,10000060.0
457470,6,770512,=,280,10000060.0
457471,6,2275365,=,280,10000060.0


In [232]:
t1_image_out = t1.drop(columns=['input_assay_id']).rename(columns={
    'input_assay_id_melloddy':'input_assay_id'
})

In [233]:
t1.shape

(404779, 5)

In [234]:
t1_image_out.to_csv('./files/T1_image_pseudolabels_full.csv', index=False)